In [5]:
####
#Author: brandon chiazza
#version 1.0
#references: 
#https://www.programiz.com/python-programming/working-csv-files
#https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
#https://realpython.com/python-boto3-aws-s3/
#CLI aws s3api create-bucket --bucket my-bucket-name --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2
#https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 
##

###Load libraries
import awscli
import selenium
import boto3
import pandas as pd
import time

from selenium import webdriver

####SCRAPE THE WEBSITE######
###call the webdriver
browser = webdriver.Chrome("C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

#identify the table to scrape
table = browser.find_element_by_css_selector('table.Bordered')

print(table)

<selenium.webdriver.remote.webelement.WebElement (session="7a103fcf9f9ce6c58994f9d764729de5", element="20e8b713-ad70-4940-8883-3084779437d7")>


In [12]:
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]
# after investigate the xml code of website, the pages are identified as the following list, click 3 means going back to the first page
pages=[1,4,5,6,7,8,3]

# loop each page
for i in pages:
    table = browser.find_element_by_css_selector('table.Bordered')

#loop through dataframe to export table
    for row in table.find_elements_by_css_selector('tr'):
        cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])
# click next page
    paginate=browser.find_element_by_xpath('/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/span[2]/a['+str(i)+']').click()
        
        
#identify the table to scrape
#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
df.dropna(inplace=True) #move invalid row
display(df) #let's have a look at the data before creating the CSV file and loading it into s3 


,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
2,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
3,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
4,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
5,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
...,...,...,...,...,...,...
102,University of Virginia Health Foundtion,40-44-88,412097394,NFP,CHARLOTTESVILLE,VA
103,Violin Player,41-40-19,270773158,NFP,EAST AMHERST,NY
104,"William A. Epps Community Center, Inc.",40-91-11,861074714,NFP,STATEN ISLAND,NY
105,WORLD SOCIETY OF CZESTOCHOWA JEWS AND THEIR DE...,40-46-49,205101779,NFP,NEW YORK,NY


In [13]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3://information-arch-kechen-assignment-8a/'#specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_group2part2' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://information-arch-kechen-assignment-8a/charities_bureau_scrape_group2part220210329033237.csv
